In [0]:
# Copy ABFS PATH
source = "abfss://source@dlsdatabrickseteteamea.dfs.core.windows.net"
bronze = "abfss://bronze@dlsdatabrickseteteamea.dfs.core.windows.net"
silver = "abfss://silver@dlsdatabrickseteteamea.dfs.core.windows.net"
gold = "abfss://gold@dlsdatabrickseteteamea.dfs.core.windows.net"

### **Test Reading data from source**

In [0]:
# from pyspark.sql.functions import current_timestamp
df = spark.read.format("parquet").load(f"{source}/orders" )
df.count()
# df = df.withColumn("test_timestamp", current_timestamp())
display(df)

### **Data Reading**

In [0]:
# Cell ที่ 3 มีการอ่านไฟล์จาก container source/orders โดยใช้ Spark Structured Streaming และ Auto Loader เพื่อเก็บ Checkpoint ของไฟล์ที่ได้อ่านแล้ว ซึ่งจะช่วยให้ไม่ต้องอ่านไฟล์เดิมซ้ำ
# readStream ใช้สำหรับการอ่านข้อมูลแบบสตรีมมิ่ง ซึ่งช่วยให้สามารถประมวลผลข้อมูลที่เข้ามาใหม่ได้อย่างต่อเนื่อง
# cloudFiles เป็นฟอร์แมตที่ใช้กับ Auto Loader ของ Databricks ซึ่งช่วยในการจัดการไฟล์ที่เข้ามาใหม่โดยอัตโนมัติ และเก็บ Checkpoint เพื่อไม่ให้เกิดการอ่านไฟล์ซ้ำ

df = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation", f"{bronze}/checkpoint_orders") \
    .load(f"{source}/orders")

# ผลลัพธ์ที่ได้จะอยู่ในโฟล์เดอร์ bronze/checkpoint_orders/_schemas.*
# df = df.withColumn("test_timestamp", current_timestamp())

### **Data Writing**

In [0]:
# .format("delta") = เขียนเป็น Delta Table (Manage Table)
# .format("parquet") = เขียนเป็นไฟล์ Delta Parquet
# .trigger(processingTime="10 seconds")    ให้เขียนทุกๆ 10 วินาที
# .trigger(once=True) เขียนเพียงครั้งเดียวแล้วจบ
parquet_stream = df.writeStream.format("parquet") \
    .outputMode("append") \
    .option("checkpointLocation" , f"{bronze}/checkpoint_orders") \
    .option("path" , f"{bronze}/orders" ) \
    .trigger(once=True) \
    .start()

# ผลลัพธ์ที่ได้คือ
# 1. Folder bronze/orders ที่มีข้อมูลเป็นไฟล์ parquet 
# 2. Folder btonze/checkpoint_orders/
#   - __tmp_path_dir.*: ไฟล์ชั่วคราวที่ใช้ในกระบวนการเขียนข้อมูล
#   - commits.*: ไฟล์ที่เก็บข้อมูลเกี่ยวกับการ commit ของแต่ละ batch ที่ถูกประมวลผล
#   - metadata: ไฟล์ที่เก็บข้อมูลเมตาของการสตรีมมิ่ง เช่น schema ของ DataFrame
#   - offsets.*: ไฟล์ที่เก็บข้อมูลเกี่ยวกับ offset ของแต่ละ batch ที่ถูกประมวลผล
#   - source.*: ไฟล์ที่เก็บข้อมูลเกี่ยวกับแหล่งข้อมูลที่ถูกอ่าน

# ถ้าเกิดมีไฟล์ใหม่เพิ่มเข้ามา ผลลัพธ์ที่ได้คือ
# 1. Folder bronze/orders ที่มีข้อมูลเป็นไฟล์ parquet ที่เพิ่มขึ้นใหม่
# 2. Folder btonze/checkpoint_orders/
#   - commits: ไฟล์ 1 เพิ่มขึ้นมา
#   - offsets: ไฟล์ 1 เพิ่มขึ้นมา
#   - source/0/rocksdb: มีการเพิ่มขึ้นมาหลายไฟล์

**การที่ Autoloader สามารถ process เฉพาะไฟล์ใหม่ที่เข้ามา เพราะว่า Rocksdb ซึ่งมีหน้าที่ในการ capture data**

**Path:	checkpoint_orders/sources/0/rocksdb**